In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#%timeit ,  line_profiler, memory_profiler

In [3]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.collocations import *
import re

In [6]:
#polar = pd.read_csv('AFINN-111.txt', sep="\t", header=None)
polar = pd.read_csv('data/senticon_es.csv')
polar.head()#.columns = ["polaritat", "valor"]

,level,orient,lemma,pos,pol,std
0,1,positive,acertado,a,0.708,0.149
1,1,positive,admirable,a,0.906,0.125
2,1,positive,admiración,n,0.450,0.331
3,1,positive,admirar,v,0.750,0.177
4,1,positive,afectivo,a,0.375,0.000


In [7]:
GEN = pd.read_csv('data/1_GEN_clean.csv')
GEN.head()
REV = pd.read_csv('data/66_REV_clean.csv')
REV.head()

,book,chapter,verse,text,version
0,GEN,1,1,"Dios, en el principio, creó los cielos y la ti...",NVI
1,GEN,1,2,"La tierra era un caos total, las tinieblas cub...",NVI
2,GEN,1,3,Y dijo Dios: «¡ Que exista la luz ! »Y la luz ...,NVI
3,GEN,1,4,Dios consideró que la luz era buena y la separ...,NVI
4,GEN,1,5,"A la luz la llamó «día», y a las tinieblas, «n...",NVI


,book,chapter,verse,text,version
0,REV,1,1,"Esta es la revelación de Jesucristo, que Dios ...",NVI
1,REV,1,2,"quien por su parte da fe de la verdad, escribi...",NVI
2,REV,1,3,Dichoso el que lee y dichosos los que escuchan...,NVI
3,REV,1,4,"Yo, Juan, escribo a las siete iglesias que est...",NVI
4,REV,1,5,"y de parte de Jesucristo, el testigo fiel, el ...",NVI


In [8]:
list_gen = GEN.text.to_list()
list_rev = REV.text.to_list()

In [9]:
list_gen[:2]
list_rev[:2]

['Dios, en el principio, creó los cielos y la tierra.',
 'La tierra era un caos total, las tinieblas cubrían el abismo, y el Espíritu de Dios se movía sobre la superficie de las aguas.']

['Esta es la revelación de Jesucristo, que Dios le dio para mostrar a sus siervos lo que sin demora tiene que suceder. Jesucristo envió a su ángel para dar a conocer la revelación a su siervo Juan, ',
 'quien por su parte da fe de la verdad, escribiendo todo lo que vio, a saber, la palabra de Dios y el testimonio de Jesucristo. ']

In [10]:
def candidates(list_sentences):
    sh_tag = [ tag_tokenizer(line) for line in(list_sentences)]
    sh_np = [t for t in(sh_tag)]
    sh_words = to_words(sh_np)
    return sh_words

In [11]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

def get_tokens(text):
    # aquest tokenizer extreurà els caracters de puntuació
    return tokenizer.tokenize(text.lower())

def token_filter(tokens):
    stop_words = nltk.corpus.stopwords.words('spanish')
    word_tokens = [word for word in tokens if word not in stop_words and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}',word)]
    return word_tokens

def tag_tokenizer(text):
    tokens = get_tokens(text)
    tokens = token_filter(tokens)
    tagged_tokens = tagged_pos(tokens)
    return tagged_tokens

def tagged_pos(tokens):
    tagged_tokens = nltk.pos_tag(tokens)
    return tagged_tokens

In [12]:
def to_words(data_list):
    ret=[]
    for line in(data_list):
        med=[]
        for w in line:
            #print(w)
            tx=''
            for el in w:
                if type(el) is tuple:
                    tx += " "+el[0]
                else:
                    tx=w[0]
            tx = tx.strip()
            med.append(tx)
        if len(med)>0:
               ret.append(med)
    return ret

In [13]:
t1 = tag_tokenizer(list_gen[0])
t1

[('dios', 'NNS'),
 ('principio', 'VBP'),
 ('creó', 'JJ'),
 ('cielos', 'NN'),
 ('tierra', 'NN')]

In [14]:
cand_gen = candidates(list_gen)

In [15]:
len(list_rev)

cand_rev = candidates(list_rev)

len(cand_rev)

382

382

In [16]:
cand_gen[:2]
cand_rev[:2]

[['dios', 'principio', 'creó', 'cielos', 'tierra'],
 ['tierra',
  'caos',
  'total',
  'tinieblas',
  'cubrían',
  'abismo',
  'espíritu',
  'dios',
  'movía',
  'superficie',
  'aguas']]

[['revelación',
  'jesucristo',
  'dios',
  'dio',
  'mostrar',
  'siervos',
  'demora',
  'suceder',
  'jesucristo',
  'envió',
  'dar',
  'conocer',
  'revelación',
  'siervo',
  'juan'],
 ['parte',
  'verdad',
  'escribiendo',
  'vio',
  'saber',
  'palabra',
  'dios',
  'testimonio',
  'jesucristo']]

In [17]:
def calculate_mean_opinions(polar, candidats):
    ret=[]
    diccio = dict(zip(polar.lemma, polar.pol))
    for i,text in enumerate(candidats):
        mitj = 0
        val = 0
        count_opinion = 0
        #print('text:'+" ".join(text))
        for j,word in enumerate(text):
            #print(word)
            if diccio.get(word) == None:
                val = 0
                #print('not in dic')
            else: 
                val = diccio.get(word)
                count_opinion += 1
            mitj += val
        #print('m='+str(mitj))
        if count_opinion:
            ret.append ( float(mitj/count_opinion))
        else:
            ret.append(0)
    print('n='+str(i))    
    return ret

In [18]:
opinions_gen = calculate_mean_opinions(polar, cand_gen)

n=1482


In [19]:
len(cand_rev)

382

In [20]:
opinions_rev = calculate_mean_opinions(polar, cand_rev)

n=381


In [21]:
opinions_gen[:5]

[0, -0.25, 0.25, 0, 0]

In [22]:
opinions_rev[:5]

[0, 0.3, -0.3125, 0, 0.2333333333333333]

In [23]:
GEN['opinions']=opinions_gen

In [24]:
GEN.head(2)

,book,chapter,verse,text,version,opinions
0,GEN,1,1,"Dios, en el principio, creó los cielos y la ti...",NVI,0.00
1,GEN,1,2,"La tierra era un caos total, las tinieblas cub...",NVI,-0.25


In [25]:
len(REV)
len(opinions_rev)

382

382

In [26]:
REV['opinions']=opinions_rev

In [33]:
bestR = REV[REV['opinions']==REV['opinions'].max()]
badR = REV[REV['opinions']==REV['opinions'].min()]
bestR
badR
bestR.text.to_list()
badR.text.to_list()

,book,chapter,verse,text,version,opinions
65,REV,3,19,Yo reprendo y disciplino a todos los que amo. ...,NVI,0.75
195,REV,12,13,"Cuando el dragón se vio arrojado a la tierra, ...",NVI,0.75
198,REV,12,16,Pero la tierra ayudó a la mujer: abrió la boca...,NVI,0.75
361,REV,21,26,Y llevarán a ella todas las riquezas y el hono...,NVI,0.75


,book,chapter,verse,text,version,opinions
335,REV,20,14,La muerte y el infierno fueron arrojados al la...,NVI,-0.8


['Yo reprendo y disciplino a todos los que amo. Por lo tanto, sé fervoroso y arrepiéntete. ',
 'Cuando el dragón se vio arrojado a la tierra, persiguió a la mujer que había dado a luz al varón. ',
 'Pero la tierra ayudó a la mujer: abrió la boca y se tragó el río que el dragón había arrojado por sus fauces. ',
 'Y llevarán a ella todas las riquezas y el honor de las naciones. ']

['La muerte y el infierno fueron arrojados al lago de fuego. Este lago de fuego es la muerte segunda. ']

In [37]:
bestG = GEN[GEN['opinions']==GEN['opinions'].max()]
badG = GEN[GEN['opinions']==GEN['opinions'].min()]
bestG
badG
bestG.text.to_list()
badG.text.to_list()

,book,chapter,verse,text,version,opinions
1149,GEN,40,16,Al ver que la interpretación había sido favora...,NVI,0.844
1171,GEN,41,16,―No so y yo quien puede hacerlo —respondió Jos...,NVI,0.844


,book,chapter,verse,text,version,opinions
660,GEN,25,26,"Luego nació su hermano, agarrado con una mano ...",NVI,-0.875


['Al ver que la interpretación había sido favorable, el jefe de los panaderos le dijo a José: ―Yo también tuve un sueño. En ese sueño, llevaba yo tres canastas de pan sobre la cabeza. ',
 '―No so y yo quien puede hacerlo —respondió José—, sino que es Dios quien le dará al faraón una respuesta favorable.']

['Luego nació su hermano, agarrado con una mano del talón de Esaú. A este lo llamaron Jacob. Cuando nacieron los mellizos, Isaac tenía sesenta años.']